In [7]:
import numpy as np
#from rich.progress import Progress

In [34]:

def normalize_matrix(non_normalized_P):
    """
    This function row-normalizes a matrix.
    This is used to convert the occupancy matrix from counts to fractional occupancies.
    """

    # Chunk-wise
    # Probably don't use this any more, there's no reason this function
    #  should opaquely expect an array of matrices.

    # n_chunks = len(non_normalized_P)
    n_bins = non_normalized_P.shape[0]

    normed_P = np.zeros(shape=(n_bins, n_bins))

    # for chunk_idx in range(n_chunks):
    for i in range(n_bins):

        if sum(non_normalized_P[i, :]) == 0:
            continue

        normed_P[i, :] = non_normalized_P[i, :] / sum(non_normalized_P[i, :])

    assert np.sum(np.isnan(normed_P)) == 0

    return normed_P

def build_occupancy_matrix(
    trajectories_to_analyze,
    time_horizon,
    absorbing,
    N_BINS,
    # N_CHUNKS,
    statesA=None,
    statesB=None,
    _skip=1,
    normalize=True,
    force_show_progress=False,
):
    """
    This function takes an input trajectory and builds the occupancy matrix for a certain time horizon.
    The 'absorbing' flag should be set to True when the matrix is being built for committor calculations,
    and to False when the goal is equilibrium distribution analysis.
    The 'statesA' and 'statesB' flags specify which sets of bins will absorb, should not be set for equilibrium
    calculations.
    '_skip' just subsamples the trajectories being analyzed.
    """

    non_normalized_P = np.zeros(shape=(N_BINS, N_BINS))

    if absorbing:
        assert statesA is not None and statesB is not None

#     with Progress(refresh_per_second=0.5, transient=True) as progress:

    for traj_idx, trajectory in enumerate(trajectories_to_analyze[::_skip]):

        start_indices = []

        if len(trajectory) == time_horizon:
            start_indices = [0]
        elif len(trajectory) > time_horizon:
            start_indices = range(len(trajectory) - time_horizon + 1)
        else:
            raise Exception(
                f"Invalid, time horizon must be less than the trajectory length ({len(trajectory)})"
            )

#         show_progress = False
#         refresh_steps = max(50, len(start_indices) // 500)
#         if len(trajectory) > 1e4 or force_show_progress:

#             traj_task = progress.add_task(
#                 "[magenta]Occupancy: Current trajectory...",
#                 total=len(start_indices),
#             )
#             show_progress = True

        for start_idx in start_indices:

#             if (
#                 start_idx % refresh_steps == 0
#                 and traj_idx % 1 == 0
#                 and show_progress
#             ):
#                 progress.update(traj_task, advance=refresh_steps)
#                 progress.refresh()

            segment = trajectory[start_idx : start_idx + time_horizon]

#                 print(segment)
            i = segment[0]

            # Check if any basin states exist in the trajectory
            # Hopefully, doing this check is faster than iterating through all segments.
            for step_idx, j in enumerate(segment):

                # Skip the first step -- this should be exactly equal to a transition matrix, per discussion
                # with Dan about including the first point on 9/18
                #                 if step_idx == 0: continue

                # To not do padding, just skip this block
                if absorbing and (j in statesA or j in statesB):

                    remaining_steps = len(segment[step_idx:])

                    # For padding
                    non_normalized_P[i, j] += remaining_steps

                    break

                else:

                    non_normalized_P[i, j] += 1

#         if show_progress:
#             progress.remove_task(traj_task)

    if normalize:
        normed_P = normalize_matrix(non_normalized_P)

        return normed_P

    else:
        return non_normalized_P

In [35]:
trajectory = [1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1,2,3,3,3,1,0,1,1,1,2,1]
trajectories = [trajectory, trajectory, trajectory, trajectory]

In [48]:
def test_build(n_iters):
    
    for i in range(n_iters):
        
        build_occupancy_matrix(trajectories, time_horizon=5
, absorbing=False, N_BINS=4, normalize=True)

In [49]:
%%time

test_build(1000)

CPU times: user 7.35 s, sys: 0 ns, total: 7.35 s
Wall time: 7.37 s
